In [1]:
import os

from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown

In [2]:
load_dotenv()

True

In [3]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

model_name = "gpt-3.5-turbo"

In [ ]:
def chain_of_thought(user_query: str):
    system_role_definition = (
        "You are a helpful assistant. Answer precisely and concisely."
    )
    user_query_complete = f"{user_query}, think step-by-step."
    messages = [
        {"role": "system", "content": system_role_definition},
        {"role": "user", "content": user_query_complete},
    ]
    response = client.chat.completions.create(model=model_name, messages=messages)
    content = response.choices[0].message.content
    return content

In [ ]:
def self_consistency_chain_of_thought(user_query: str, number_of_chains: int = 3):
    # run CoT multiple times
    res = []
    for i in range(1, number_of_chains + 1):
        res_chain = chain_of_thought(user_query=user_query)
        res.append(res_chain)
        print(f"Chain {i}: {res_chain}")
        print("-" * 100)

    # ask the model to return most common answer
    res_concat = ";".join(res)
    self_consistency_prompt = f"You will get multiple answers in <<>>, separated by ; <<{res_concat}>> Extract only the final equations and return the most common equation as it was provided originally."
    messages = [{"role": "user", "content": f"{self_consistency_prompt}"}]
    response = client.chat.completions.create(model=model_name, messages=messages)
    content = response.choices[0].message.content
    return content

In [4]:
user_prompt = "The goal of the Game of 24 is to use the four arithmetic operations (addition, subtraction, multiplication, and division) to combine four numbers and get a result of 24. The numbers are 3, 4, 6, and 8. It is mandatory to use all four numbers. Please check the final equation for correctness. Hints: Identify the basic operations, Prioritize multiplication and division, Look for combinations that make numbers divisible by 24, Consider order of operations, Use parentheses strategically, Practice with different number combinations"
res = self_consistency_chain_of_thought(user_query=user_prompt, number_of_chains=5)

Chain 1: Sure, one possible solution for getting 24 using the numbers 3, 4, 6, and 8 is: 

$$(8 ÷ (4 - (6 ÷ 3))) = 24$$
----------------------------------------------------------------------------------------------------
Chain 2: Sure! Here's a possible solution using the numbers 3, 4, 6, and 8:

$((6/3) * (8+4))$

Let's simplify this to confirm if it equals 24:

1. $6/3 = 2$
2. $8+4 = 12$
3. $2 * 12 = 24$

Therefore, $((6/3) * (8+4))$ equates to 24.
----------------------------------------------------------------------------------------------------
Chain 3: Sure! Following these guidelines, one possible correct equation to achieve the result of 24 using the numbers 3, 4, 6, and 8 is: (6/(1-(3/8))) * 4.
----------------------------------------------------------------------------------------------------
Chain 4: Sure! Here's a possible equation that uses the numbers 3, 4, 6, and 8 to get a result of 24:

$$(8 * 3) - 6 + 4 = 24$$

Feel free to explore different combinations and operation

In [5]:
Markdown(res)

Sure, the most common final equation provided originally is: $((6/3) * (8+4)) = 24$